In [1]:
import os
import sys
import cv2
import json
import shutil
from glob import glob
from tqdm import tqdm
from pathlib import Path

from utils.table2label import table2layout, fuse_gt_info, judge_error
from utils.table_helper import correct_table



def table2label(table_dir, label_dir, error_file_path):
    table_error = {}
    json_files = sorted(glob(os.path.join(table_dir, '*.json')))
    # json_files = sorted(glob(os.path.join(table_dir, '*.json')))[7087:]
    for idx, json_path in tqdm(enumerate(json_files), total=len(json_files)):
        json_name = os.path.basename(json_path)
        json_id = json_name.split('.')[0]
        table = json.load(open(json_path, 'r'))

        # table = correct_table(table)
        gt_label = table2layout(table)

        if judge_error(table, gt_label):
            table_error[json_name] = 'table json error'
            continue

        ## 有线表格得到的bbox还是cell框，不是text框
        gt_label = fuse_gt_info(gt_label, table)

        gt_json_path = os.path.join(label_dir, f'{json_id}-gt.json')
        json.dump(gt_label, open(gt_json_path, 'w'), indent=4)
    json.dump(table_error, open(error_file_path, 'w'), indent=4)

    print('table error: {}'.format(len(table_error)))

## STEP.1 gen_gt_labels

In [2]:
'''
输入
    训练集目录: {dataset_root}/train
输出
    训练集标注目录: {dataset_root}/train_gt_json/
    {dataset_root}/train_error.json

使用 fix_gt_table 前 报错的：684
使用 fix_gt_table 后 报错的：665
    + 清除area==0的数据：664
'''
DATASET = "train_jpg480max"
DATASET_ROOT = '/media/ubuntu/Date12/TableStruct/new_data'
TABLE_DIR = os.path.join(DATASET_ROOT, DATASET)
LABEL_DIR = os.path.join(DATASET_ROOT, f'{DATASET}_gt_json')
TABLE_ERROR_PATH = os.path.join(DATASET_ROOT, f'{DATASET}_error.json')

if os.path.exists(LABEL_DIR):
    shutil.rmtree(LABEL_DIR)
Path.mkdir(Path(LABEL_DIR), parents=True)

table2label(TABLE_DIR, LABEL_DIR, TABLE_ERROR_PATH)


100%|██████████| 12104/12104 [04:48<00:00, 41.94it/s] 

table error: 688
